# <div align="center" style="color: #ff5733;">TSA Closure Second Notification</div>

In [23]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [24]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID ae9087a4-919a-4655-bea6-0c7d630ac98f successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-07-30,630514


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [25]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\May24&June24 Ongoing Batch\SecondNotification_files\Type 1 & 2.xlsx", sheet_name='Sheet1')
d1.shape

(14320, 2)

In [26]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	14320


,Customer_Id,ACTION_TYPE
0,2371220,TYPE 1
1,2502195,TYPE 1
2,2499439,TYPE 1
3,2451802,TYPE 1
4,2462375,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [28]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\May24&June24 Ongoing Batch\SecondNotification_files\Type 3.xlsx", sheet_name='Sheet1')
d2.shape

(1394, 2)

In [29]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	1394


,Customer_Id,ACTION_TYPE
0,2342057,TYPE 3
1,1017332,TYPE 3
2,2141357,TYPE 3
3,1829218,TYPE 3
4,1015223,TYPE 3


In [30]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(15714, 2)

In [31]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    14290
TYPE 3     1394
TYPE 2       30
Name: count, dtype: int64

In [35]:
result[result['Customer_Id'] == '2256481']

,Customer_Id,ACTION_TYPE


In [33]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15714 entries, 0 to 15713
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  15714 non-null  int64 
 1   ACTION_TYPE  15714 non-null  object
dtypes: int64(1), object(1)
memory usage: 245.7+ KB


In [34]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15714 entries, 0 to 15713
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  15714 non-null  object
 1   ACTION_TYPE  15714 non-null  object
dtypes: object(2)
memory usage: 245.7+ KB


In [36]:
result.to_csv("temp.csv", index = False)

In [37]:
a = tuple(result['Customer_Id'])
a[:5]

('2371220', '2502195', '2499439', '2451802', '2462375')

In [38]:
d = '2024-07-31'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-05-08'
# Calculate D-180 days
d_minus_180_days = '2024-01-09'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-07-30 00:00:00
D-60 day from the current date - 1: 2024-05-08
D-180 day from the current date -1: 2024-01-09
D-1 day from the current date: 2024-07-30
D-60 day from the current date: 2024-05-08
D-180 day from the current date: 2024-01-09


The date should be D-1? If the date is not D-1 then ask Bala to refresh the table.

Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [39]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.shape

Job ID 482d319c-8ce0-4f22-b124-08708eec2015 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(15712, 29)

In [40]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (Active_Loan = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [41]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-07-30'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in ('2371220', '2502195', '2499439', '2451802', '2462375', '2452433', '2497108', '2496598', '2444336', '2495373', '2456956', '2476578', '2442697', '2495329', '2472111', '2484651', '2448096', '2491779', '2462089', '2463309', '2447013', '2454241', '2458711', '2476027', '2476104', '2485651', '2481141', '2501907', '2447691', '2466247', '2492548', '2475802', '2477131', '2468323', '2481063', '2462508', '2462034', '2455877',

In [42]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID f5121555-1654-422c-8c64-25c1eef697a1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [43]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
14594,2491357,2024-05-01,60824913570007,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-05-01,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,0,NaT,NaT,TYPE 1
8616,2454046,2024-04-06,60824540460007,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-06,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,0,NaT,NaT,TYPE 1
3638,2480818,2024-04-25,60824808180006,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-25,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,1,NaT,2024-07-25,TYPE 1
10010,2467501,2024-04-16,60824675010006,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-22,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,1,NaT,2024-07-16,TYPE 1
2466,2451850,2024-04-04,60824518500005,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-07-09,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,1,NaT,2024-07-04,TYPE 1
11304,2487603,2024-04-29,60824876030001,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-29,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,1,NaT,2024-07-29,TYPE 1
10703,2447523,2024-04-01,60824475230002,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-01,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,1,NaT,2024-07-01,TYPE 1
14765,2485518,2024-04-28,60824855180007,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-28,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,1,NaT,2024-07-27,TYPE 1
5121,2477768,2024-04-23,60824777680006,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-07-09,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,0,NaT,NaT,TYPE 1
13088,2484242,2024-04-27,60824842420004,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-04-27,NaT,NaT,NaN,2024-07-30,Batch_1,July,0,1,NaT,2024-07-27,TYPE 1


In [44]:
df.shape

(15712, 32)

In [45]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       14149
TYPE 3        1328
NO ACTION      207
TYPE 2          28
Name: count, dtype: int64

In [46]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\May24&June24 Ongoing Batch\SecondNotification_files\20240731_TSA_Account_Closure_2ndtNotification.xlsx", sheet_name='TSA_Closure_2ndNotification', index = False)